# Archive and Copy Clips into Distribution Folder

Follow instructions to move clips into the archive and copy them into the distribution folder.

## Step 1

Run the cell below.

You will be prompted to follow a link to authorize access to the Drive. Click the link and make sure that you sign in as ptrstoryteam@gmail.com. Copy the authorization token, return to this page, paste the token into the provided box, and hit enter on your keyboard. This completes the authorization.

In [1]:
# LIBRARIES

import os
import shutil
import glob
import csv
from datetime import datetime

from google.colab import drive
import pandas as pd

# MOUNTING THE DRIVE/AUTHORIZATION

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Step 2

Run the cell below to complete initial setup.

In [37]:
###############################################################################
# DIRECTORIES
###############################################################################

# Path to root directory
PATH_ROOT = "/content/drive/My Drive/Protect the Results Story Team/10 PTR DISTRIBUTION/TEST/"

# Name of/path to upload folder
NAME_UPLOAD = "UPLOAD/"
PATH_UPLOAD = PATH_ROOT + NAME_UPLOAD

# Name of/path to archive folder
NAME_ARCHIVE = "ARCHIVE/"
PATH_ARCHIVE = PATH_ROOT + NAME_ARCHIVE

# Name of/path to distribution folder
NAME_DISTRIBUTION = "DISTRIBUTION/"
PATH_DISTRIBUTION = PATH_ROOT + NAME_DISTRIBUTION

# Name of/path to metadata folder
NAME_METADATA = "METADATA/"
PATH_METADATA = PATH_ROOT + NAME_METADATA

###############################################################################
# FILES
###############################################################################

# Name of master metadata CSV
NAME_MASTER_CSV = "master_csv_test.csv"

###############################################################################
# DICTIONARIES
###############################################################################

blue_city_names = {
    "Boston" : "Boston",
    "Chicago" : "Chicago",
    "LosAngeles" : "Los Angeles",
    "NewYork" : "New York City",
    "Portland" : "Portland",
    "SanFrancisco" : "San Francisco (East Bay)",
    "Seattle" : "Seattle",
    "DC" : "Washington, D.C."
}

blue_state_names = {
    "ca" : "California",
    "co" : "Colorado",
    "ct" : "Connecticut",
    "de" : "Delaware",
    "hi" : "Hawaii",
    "il" : "Illinois",
    "me" : "Maine",
    "md" : "Maryland",
    "ma" : "Massachusetts",
    "nv" : "Nevada",
    "nh" : "New Hampshire",
    "nj" : "New Jersey",
    "nm" : "New Mexico",
    "ny" : "New York",
    "or" : "Oregon",
    "ri" : "Rhode Island",
    "vt" : "Vermont",
    "va" : "Virginia",
    "wa" : "Washington"
}

red_state_names = {
    "al" : "Alabama",
    "ak" : "Alaska",
    "ar" : "Arkansas",
    "id" : "Idaho", 
    "in" : "Indiana",
    "ia" : "Iowa",
    "ks" : "Kansas",
    "ky" : "Kentucky",
    "la" : "Louisiana",
    "ms" : "Mississippi",
    "mo" : "Missouri",
    "mt" : "Montana",
    "ne" : "Nebraska",
    "nd" : "North Dakota",
    "ok" : "Oklahoma",
    "sc" : "South Carolina",
    "tn" : "Tennessee",
    "ut" : "Utah",
    "wv" : "West Virginia",
    "wy" : "Wyoming"
}

swing_state_names = {
    "az" : "Arizona",
    "fl" : "Florida",
    "ga" : "Georgia",
    "mi" : "Michigan",
    "mn" : "Minnesota",
    "nc" : "North Carolina",
    "oh" : "Ohio",
    "pa" : "Pennsylvania",
    "tx" : "Texas",
    "wi" : "Wisconsin"
}

# Dictionary of dictionaries
project_dictionaries = {
    "bluecities" : blue_city_names,
    "bluestates" : blue_state_names,
    "redstates" : red_state_names,
    "swingstates" : swing_state_names
}

###############################################################################
# FUNCTIONS
###############################################################################

# Reads and returns the master metadata CSV
def read_master_csv():
  return(pd.read_csv(PATH_METADATA + NAME_MASTER_CSV))

# Converts string representation of list to literal list
def string_to_list(string_list):
  str_list = string_list.strip('][').split(', ')
  return([string.strip('\'') for string in str_list])

# Returns the path where the clip belongs in the archive
def get_clip_path(filename, metadata):
  # Fields are:
  # 0: date (YYMMDD)
  # 1: city name (standardized only for big blue cities)
  # 2: state (2-character postal code)
  # 3: broll or int (b-roll or interview)
  # 4: clip's unique hash ID
  fields = filename.split("_")

  # Read in which project clip belongs to
  project = metadata[metadata["hash_id" == fields[4]]]["project_id"].values[0]

  # Also need to read in whether the clip contains sensitive footage
  clip_tags = string_to_list(list(metadata[metadata["hash_id"] == fields[4]]["tags"].values)[0])

  # Build clip path: project/location/(sensitive or general)
  path_list = []
  path_list[0] = project

  location = fields[1] if project == "bluecities" else fields[2]
  path_list[1] = project_dictionaries[project][location]

  path_list[2] = "sensitive" if "sensitive" in clip_tags else "general"

  path_clip = "/".join(path_list)

  return(PATH_ARCHIVE + path_clip)

'two'